<a href="https://colab.research.google.com/github/vatsmanish/Handwritten_ocr_experiment/blob/master/Inception_AV_EXP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.layers.core import Layer
import keras.backend as K
import tensorflow as tf
from keras.datasets import cifar10

In [0]:
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

import cv2 
import numpy as np 
from keras.datasets import cifar10 
from keras import backend as K 
from keras.utils import np_utils

import math 
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import pickle
import csv

from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K


# Import Keras libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:
img_width, img_height = 224,224

epochs = 10
batch_size = 150
n_classes = 26

In [0]:
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3) 

In [0]:
K.set_learning_phase(1)

In [0]:
# path_base="/content/drive/My Drive/results/train/train_letters/"
# #path_base="/content/drive/My Drive/test_letters (1)"
# val_base="/content/drive/My Drive/test_letters (1)"

In [0]:
# path_base="/content/drive/My Drive/test_letters (1)"
# val_base="/content/drive/My Drive/results/test/test_letters_19"

In [0]:
path_train = "/content/drive/My Drive/colab_notebooks/final merge 2 & 3"
# path_test = "/content/gdrive/My Drive/colab_notebooks/test upload/model/test"
path_test = "/content/drive/My Drive/colab_notebooks/test_scanned 1"

In [0]:
train_data_dir = path_train
validation_data_dir = path_test

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 10,
    width_shift_range=0.2,
    height_shift_range=0.2)
    #horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical')

Found 16944 images belonging to 50 classes.
Found 7925 images belonging to 50 classes.


In [0]:
X_train_new=train_generator[0][0]
Y_train_new=train_generator[0][1]

In [0]:
X_test_new=validation_generator[0][0]
Y_test_new=validation_generator[0][1]

In [0]:
train_generator[0][0].shape

(1, 224, 224, 3)

In [0]:
train_generator[0][1].shape

(1, 50)

In [0]:
train_generator[0][1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]], dtype=float32)

In [0]:
nb_train_samples = 5355
nb_validation_samples = 5355

In [0]:
#num_classes = 26

def load_our_data_train_X(img_rows, img_cols):

    # Load our training and validation sets
    X_train_new=train_generator[0][0]
    #Y_train_new=train_generator[0][1]

    #X_valid_new=validation_generator[0][0]
    #Y_valid_new=validation_generator[0][1]

    # Resize training images
    X_train_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train_new[:,:,:,:]])
    #X_valid_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid_new[:,:,:,:]])

    # Transform targets to keras compatible format

    #Y_train_new =np_utils.to_categorical(Y_train_new, 26)
    #Y_valid_new = np_utils.to_categorical(Y_valid_new, 26)
    
    X_train_new = X_train_new.astype('float32')
    #X_valid_new = X_valid_new.astype('float32')

    # preprocess data
    #X_train_new = X_train_new / 255.0
    #X_valid_new = X_valid_new / 255.0

    return X_train_new

In [0]:
#num_classes = 26

def load_our_data_train_Y(img_rows, img_cols):

    # Load our training and validation sets
    #X_train_new=train_generator[0][0]
    Y_train_new=train_generator[0][1]

    #X_valid_new=validation_generator[0][0]
    #Y_valid_new=validation_generator[0][1]

    # Resize training images
    #X_train_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train_new[:,:,:,:]])
    #X_valid_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid_new[:,:,:,:]])

    # Transform targets to keras compatible format

    #Y_train_new =np_utils.to_categorical(Y_train_new, 26)
    #Y_valid_new = np_utils.to_categorical(Y_valid_new, 26)
    
    #X_train_new = X_train_new.astype('float32')
    #X_valid_new = X_valid_new.astype('float32')

    # preprocess data
    #X_train_new = X_train_new / 255.0
    #X_valid_new = X_valid_new / 255.0

    return  Y_train_new

In [0]:
#num_classes = 26

def load_our_data_test_Y(img_rows, img_cols):

    # Load our training and validation sets
    #X_train_new=train_generator[0][0]
    #Y_train_new=train_generator[0][1]

    #X_valid_new=validation_generator[0][0]
    Y_valid_new=validation_generator[0][1]

    # Resize training images
    #X_train_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train_new[:,:,:,:]])
    #X_valid_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid_new[:,:,:,:]])

    # Transform targets to keras compatible format
    #Y_train_new = np_utils.to_categorical(Y_train_new, 26)
    #Y_valid_new = np_utils.to_categorical(Y_valid_new, 26)
    
    #X_train_new = X_train_new.astype('float32')
   # X_valid_new = X_valid_new.astype('float32')

    # preprocess data
    #X_train_new = X_train_new / 255.0
    #X_valid_new = X_valid_new / 255.0

    return Y_valid_new

In [0]:
#num_classes = 26

def load_our_data_test_X(img_rows, img_cols):

    # Load our training and validation sets
    #X_train_new=train_generator[0][0]
    #Y_train_new=train_generator[0][1]

    X_valid_new=validation_generator[0][0]
    #Y_valid_new=validation_generator[0][1]

    # Resize training images
    #X_train_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train_new[:,:,:,:]])
    X_valid_new = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid_new[:,:,:,:]])

    # Transform targets to keras compatible format
    #Y_train_new = np_utils.to_categorical(Y_train_new, 26)
    #Y_valid_new = np_utils.to_categorical(Y_valid_new, 26)
    
    #X_train_new = X_train_new.astype('float32')
    X_valid_new = X_valid_new.astype('float32')

    # preprocess data
    #X_train_new = X_train_new / 255.0
    #X_valid_new = X_valid_new / 255.0

    return X_valid_new

In [0]:
X_train_new=np.empty(shape=[10,224,224,3])

for i in range(1600):
    X_train_new1 = load_our_data_train_X(224, 224)
    X_train_new=np.append(X_train_new,X_train_new1,axis=0)
    #Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
for i in range(500):
    X_train_new1 = load_our_data_train_X(224, 224)
    X_train_new=np.append(X_train_new,X_train_new1,axis=0)
    #Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
for i in range(500):
    X_train_new1 = load_our_data_train_X(224, 224)
    X_train_new=np.append(X_train_new,X_train_new1,axis=0)
    #Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
for i in range(500):
    X_train_new1 = load_our_data_train_X(224, 224)
    X_train_new=np.append(X_train_new,X_train_new1,axis=0)
    #Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:

Y_train_new=np.empty(shape=[10,50])
for i in range(1600):
    Y_train_new1 = load_our_data_train_Y(224, 224)
    Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
for i in range(500):
    Y_train_new1 = load_our_data_train_Y(224, 224)
    Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
for i in range(500):
    Y_train_new1 = load_our_data_train_Y(224, 224)
    Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
# for i in range(500):
#     Y_train_new1 = load_our_data_train_Y(224, 224)
#     Y_train_new=np.append(Y_train_new,Y_train_new1,axis=0)

In [0]:
X_test_new=np.empty(shape=[10,224,224,3])

for i in range(790):
    X_test_new1 = load_our_data_test_X(224, 224)
    X_test_new=np.append(X_test_new,X_test_new1,axis=0)
    #Y_test_new=np.append(Y_test_new,Y_test_new1,axis=0)

In [0]:

Y_test_new=np.empty(shape=[10,26])
for i in range(790):
    Y_test_new1 = load_our_data_test_Y(224, 224)
    X_test_new=np.append(X_test_new,X_test_new1,axis=0)
    # Y_test_new=np.append(Y_test_new,Y_test_new1,axis=0)

In [0]:
def inception_module(x,
                     filters_1x1,
                     filters_3x3_reduce,
                     filters_3x3,
                     filters_5x5_reduce,
                     filters_5x5,
                     filters_pool_proj,
                     name=None):
    
    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    
    conv_3x3 = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_3x3)

    conv_5x5 = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(x)
    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(conv_5x5)

    pool_proj = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', kernel_initializer=kernel_init, bias_initializer=bias_init)(pool_proj)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)
    
    return output

In [0]:
kernel_init = keras.initializers.glorot_uniform()
bias_init = keras.initializers.Constant(value=0.2)

In [0]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2', kernel_initializer=kernel_init, bias_initializer=bias_init)(input_layer)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)
x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=64,
                     filters_3x3_reduce=96,
                     filters_3x3=128,
                     filters_5x5_reduce=16,
                     filters_5x5=32,
                     filters_pool_proj=32,
                     name='inception_3a')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=192,
                     filters_5x5_reduce=32,
                     filters_5x5=96,
                     filters_pool_proj=64,
                     name='inception_3b')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=192,
                     filters_3x3_reduce=96,
                     filters_3x3=208,
                     filters_5x5_reduce=16,
                     filters_5x5=48,
                     filters_pool_proj=64,
                     name='inception_4a')


x1 = AveragePooling2D((5, 5), strides=3)(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(26, activation='softmax', name='auxilliary_output_1')(x1)

x = inception_module(x,
                     filters_1x1=160,
                     filters_3x3_reduce=112,
                     filters_3x3=224,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4b')

x = inception_module(x,
                     filters_1x1=128,
                     filters_3x3_reduce=128,
                     filters_3x3=256,
                     filters_5x5_reduce=24,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4c')

x = inception_module(x,
                     filters_1x1=112,
                     filters_3x3_reduce=144,
                     filters_3x3=288,
                     filters_5x5_reduce=32,
                     filters_5x5=64,
                     filters_pool_proj=64,
                     name='inception_4d')


x2 = AveragePooling2D((5, 5), strides=3)(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(26, activation='softmax', name='auxilliary_output_2')(x2)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_4e')

x = MaxPool2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)

x = inception_module(x,
                     filters_1x1=256,
                     filters_3x3_reduce=160,
                     filters_3x3=320,
                     filters_5x5_reduce=32,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5a')

x = inception_module(x,
                     filters_1x1=384,
                     filters_3x3_reduce=192,
                     filters_3x3=384,
                     filters_5x5_reduce=48,
                     filters_5x5=128,
                     filters_pool_proj=128,
                     name='inception_5b')

x = GlobalAveragePooling2D(name='avg_pool_5_3x3/1')(x)

x = Dropout(0.4)(x)

x = Dense(26, activation='softmax', name='output')(x)

In [0]:
model = Model(input_layer, [x, x1, x2], name='inception_v1')

In [0]:
model.summary()

Model: "inception_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv_1_7x7/2 (Conv2D)           (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pool_1_3x3/2 (MaxPooling2D) (None, 56, 56, 64)   0           conv_1_7x7/2[0][0]               
__________________________________________________________________________________________________
conv_2a_3x3/1 (Conv2D)          (None, 56, 56, 64)   4160        max_pool_1_3x3/2[0][0]           
_______________________________________________________________________________________

In [0]:
epochs = 50
initial_lrate = 0.01

def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

sgd = SGD(lr=initial_lrate, momentum=0.9, nesterov=False)

lr_sc = LearningRateScheduler(decay, verbose=1)

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'], loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])

In [0]:
history = model.fit(X_train_new, [Y_train_new, Y_train_new, Y_train_new],
                    validation_data=(X_test_new, [Y_test_new, Y_test_new, Y_test_new]), epochs=epochs, batch_size=256, callbacks=[lr_sc])

ValueError: ignored

In [0]:
X_test_new   Y_train_new

In [0]:
history = model.fit(train_generator[0][0], [train_generator[0][1], train_generator[0][1], train_generator[0][1]], validation_data=(validation_generator[0][0], [validation_generator[0][1], validation_generator[0][1], validation_generator[0][1]]), epochs=epochs, batch_size=256, callbacks=[lr_sc])

ValueError: ignored

In [0]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=epochs, callbacks=[lr_sc])